In [1]:
from model2 import *

In [2]:
traj = md.load('../../xtc_files/50000eql2.xtc', top='../../xtc_files/conf.gro')

In [3]:
n_frames,n_atoms,n_mols = (0,0,0)
n_neigh = 15

In [4]:
model_1_name = "Intra.pt"
model_stage1 = torch.load("./models/"+model_1_name)
model_stage1

VGAE(
  (encoder): VariationalGCNEncoder(
    (conv1): GATConv(5, 15, heads=3)
    (head_transform1): Linear(45, 15, bias=True)
    (bn1): BatchNorm(15)
    (conv2): GCNConv(15, 9)
    (bn2): BatchNorm(9)
    (linear1): Linear(9, 100, bias=True)
    (linear2): Linear(100, 4, bias=True)
    (transform): Linear(16, 3, bias=True)
    (mu): Linear(3, 3, bias=True)
    (logstd): Linear(3, 3, bias=True)
  )
  (decoder): VariationalGCNDecoder(
    (inv_transform): Linear(3, 4, bias=True)
    (conv1): GCNConv(1, 9)
    (bn1): BatchNorm(9)
    (conv2): GCNConv(9, 3)
    (bn2): BatchNorm(3)
    (conv3): GCNConv(3, 3)
    (linear1): Linear(3, 512, bias=True)
    (linear2): Linear(512, 128, bias=True)
    (linear3): Linear(128, 5, bias=True)
  )
)

In [5]:
load = False
if load:
    graphs_cry_cpu = torch.load(f'./graphs/simple/crystal_neigh{n_neigh}.pt')
    graphs_melt_cpu = torch.load(f'./graphs/simple/melt_neigh{n_neigh}.pt')
else:
    # all_frames_cry = md.load_xtc("../simulation/para_crystal.xtc", top="../simulation/para_crystal.gro")
    all_frames_melt= md.load_xtc('../../xtc_files/50000eql2.xtc', top='../../xtc_files/conf.gro')
    # all_frames_melt = md.load_xtc("../simulation/para_melt.xtc", top="../simulation/para_melt.gro")
    # all_frames_cry.xyz = all_frames_cry.xyz*10
    all_frames_melt.xyz = all_frames_melt.xyz*10
    # all_frames_cry = all_frames_cry
    all_frames_melt = all_frames_melt[::50]
    global n_frames,n_atoms,n_mols 
    # n_frames = all_frames_cry.xyz.shape[0]
    # n_atoms = all_frames_cry.topology.residue(0).n_atoms
    # n_mols = all_frames_cry.topology.n_residues
    # condensed_crystal = condenseAllFrames(all_frames_cry)
#     condensed_crystal = condenseAllFrames_same_res_id(f,n_frames)
    condensed_melt = condenseAllFrames(all_frames_melt)
    # graphs_cry_cpu = get_graphs(condensed_crystal,"cry")
    graphs_melt_cpu = get_graphs(condensed_melt,"melt")
#     graphs_cry_cpu = pad(graphs_cry_cpu)
#     graphs_melt_cpu = pad(graphs_melt_cpu)
    # torch.save(graphs_cry_cpu, f'./graphs/simple/crystal_neigh{n_neigh}.pt')
    torch.save(graphs_melt_cpu,f'./graphs/simple/melt_neigh{n_neigh}.pt')

  0%|          | 0/1001 [00:00<?, ?it/s]


NameError: name 'model_stage1' is not defined

In [ ]:
test_frame = md.load_xtc("../simulation/para_crystal.xtc", top="../simulation/para_crystal.gro",frame=0)


FileNotFoundError: [Errno 2] No such file or directory: '../simulation/para_crystal.gro'

In [ ]:
cf = condenseFrame2(test_frame)

NameError: name 'test_frame' is not defined

In [ ]:
# %matplotlib widget

g = get_graph(cf,10,n_neigh,"cry")
plot3D(g)

NameError: name 'cf' is not defined

In [ ]:
graphs_cpu = graphs_cry_cpu + graphs_melt_cpu

NameError: name 'graphs_cry_cpu' is not defined

In [ ]:
print(len(graphs_cry_cpu))
print(len(graphs_melt_cpu))

NameError: name 'graphs_cry_cpu' is not defined

In [ ]:
# shuffle the graphs
np.random.shuffle(graphs_cpu)

NameError: name 'graphs_cpu' is not defined

In [ ]:
import os
in_channels = graphs_cpu[0].num_features
out_channels = 5
lr = 1e-4
n_epochs = 400
batch_size=64
test_train_split = 0.8
model_name = "LOCAL_Classifier.pt"
model_loaded = False
force_train = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if os.path.exists("./"+model_name) and not force_train:
    model = torch.load("./"+model_name)
    model_loaded = True
else:
    model = VGAE_S2(VariationalGCNEncoder_S2(in_channels, out_channels,batch_size,n_neigh+1),
                    VariationalGCNDecoder_S2(out_channels, in_channels,batch_size,n_neigh+1))

model = model.to(device)
model = model.double()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=300, gamma=0.5)


NameError: name 'graphs_cpu' is not defined

In [ ]:
graphs_device = []
for graph in tqdm(graphs_cpu):
    graphs_device.append(graph.to(device))

NameError: name 'graphs_cpu' is not defined

In [ ]:
graphs = graphs_device
total_graphs = len(graphs)

In [ ]:
print("Using:",device )
print("total_graphs:", total_graphs)
print("batch Size:" , batch_size)
print("number of batches:",total_graphs//batch_size)
print("Neighbours:",n_neigh)
print("Model\n",model)

NameError: name 'device' is not defined

In [ ]:
split = int(test_train_split * total_graphs)

train_loader = DataLoader(graphs[:split], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(graphs[split:], batch_size=batch_size, shuffle=True)

NameError: name 'test_train_split' is not defined

In [ ]:
cross_entropy_loss = torch.nn.CrossEntropyLoss()
MSE_loss = torch.nn.MSELoss()

def train():
    model.train()
    model.zero_grad()
    all_loss_mse = 0
    for data in train_loader:
        optimizer.zero_grad()
        
        z,encoded_edge_index = model.encode(data.x, data.edge_index)
        decoded_x, decoded_edge_index = model.decode(z, encoded_edge_index)

        
        loss = MSE_loss(decoded_x, data.x)


        loss.backward()
        all_loss_mse += float(loss)
        optimizer.step()


    return all_loss_mse/len(train_loader)

In [ ]:
def test():
    model.eval()
    model.zero_grad()
    all_loss_mse = 0
    for data in test_loader:
        optimizer.zero_grad()
        z,encoded_edge_index = model.encode(data.x, data.edge_index)
        decoded_x, decoded_edge_index = model.decode(z, encoded_edge_index)

        loss = MSE_loss(decoded_x, data.x)

        all_loss_mse += float(loss)


    return all_loss_mse/len(test_loader)

In [ ]:
train_losses = []
test_losses = []

if model_loaded:
    print("Pretrained Model Loaded, no training required")
else:
    for epoch in range(1, n_epochs + 1):

        train_loss_mse = train()
        test_loss_mse = test()

        print(f"Epoch: {epoch:03d}")
        print(f'\tTrain:\tMSE Loss: {train_loss_mse:.4f}')
        print(f'\tTest: \tMSE Loss: {test_loss_mse:.4f}')

        train_losses.append([train_loss_mse]);
        test_losses.append([test_loss_mse]);

        scheduler.step()
    torch.save(model,"./"+model_name)

NameError: name 'model_loaded' is not defined

In [ ]:
if not model_loaded:
    plt.figure()
    plt.plot(test_losses, label='Total Test Loss')
    plt.plot(train_losses, label='Total Train Loss')
    plt.legend()
    plt.title('Total Loss')
    plt.show()

NameError: name 'model_loaded' is not defined

In [ ]:
test_loader_cry = DataLoader(graphs_cry_cpu[::100], batch_size=1)
pred_cry=[]
for data in tqdm(test_loader_cry):
    model.eval()
    data = data.to(device)
    pred = model(data.x.float(),data.edge_index)
    pred_cry.append(pred.cpu().detach().numpy())

pred_cry = np.array(pred_cry)


NameError: name 'graphs_cry_cpu' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_test = test_loader_melt
cm = confusion_matrix(y_test, predictions)
ConfusionMatrixDisplay(cm).plot()


NameError: name 'test_loader_melt' is not defined

In [ ]:
# %matplotlib widget

fig = plt.figure(figsize=(15,4))
data_melt = pred_melt
data_cry = pred_cry

plt.scatter(data_melt[:,0],data_melt[:,1],c="r",label="melt")
plt.scatter(data_cry[:,0],data_cry[:,1],c="b",label="crystal")
plt.legend()
plt.show()

NameError: name 'pred_melt' is not defined

<Figure size 1500x400 with 0 Axes>

In [ ]:
n_sample = 1000
n_class = 2
n_1 = 1000
n_2 = 1000